In [14]:
import pandas
import mido
import os
import re
import shutil
from multiprocessing import Pool


In [ ]:
# Use this to strip titles and instrument data of non-alphabetic characters
pattern = "[^a-zA-Z]+"

In [35]:
import os
os.getcwd()
raw_data_folder = os.path.join(os.pardir, "raw_data", "Unzipped")
#raw_data_folder = os.path.join(os.pardir, "test_data")
output_folder = os.path.join(os.pardir, "data_test")
raw_data_folder

'../raw_data/Unzipped'

In [38]:
class FileProcessor():
    def __init__(self):
        self.saved_songs = []
        self.errors = 0
        self.num_processed = 0
        self.already_saved = os.listdir(output_folder)

    def parse_file(self, child):
        self.num_processed += 1
        if self.num_processed % 500 == 0:
            print(f"Processing file number: {self.num_processed}")

        # If the path is a midi file, load the file
        child_path = os.path.join(raw_data_folder, child)
        try:
            if child_path.endswith(".mid") and child not in self.already_saved:
                song_path = child_path
                midi_file = mido.MidiFile(song_path, clip=True)
                dg_song = self.continueidentify_drum_and_guitar(midi_file)
                if dg_song:
                    title = re.sub(pattern, "", child[:-4]).lower()
                    if title not in saved_songs:
                        self.saved_songs.append(title)
                        #midi_file.save(f'{output_folder}/{child}')
                        shutil.copyfile(song_path, f'{output_folder}/{child}')


            # If the path is a folder, load the midi files inside
            elif os.path.isdir(child_path):
                for child in os.listdir(child_path):
                    song_path = os.path.join(child_path, child)
                    if song_path.endswith(".mid") and child not in self.already_saved:
                        midi_file = mido.MidiFile(song_path, clip=True)
                        dg_song = self.identify_drum_and_guitar(midi_file)
                        if dg_song:
                            title = re.sub(pattern, "", child[:-4]).lower()
                            if title not in saved_songs:
                                self.saved_songs.append(title)
                                #midi_file.save(f'{output_folder}/{child}')
                                shutil.copyfile(song_path, f'{output_folder}/{child}')

        except Exception as ke:
            self.errors +=1


    def identify_drum_and_guitar(self, midi):
        guitars = {"guitar", "guita", "gtr", "egtr", "eguitar"}
        drums = {"drums", "drum", "drumsperc", "perc", "percussion"}

        pattern = "[^a-zA-Z ]+"
        has_drum = False
        has_guitar = False

        for track in midi:
            # Check if instruments in track name
            if track.type == "track_name":
                words = {re.sub(pattern, '', word.lower()) for word in track.name.split()}
                if words.intersection(guitars):
                    has_guitar = True
                if words.intersection(drums):
                    has_drum = True
        if has_drum and has_guitar:
            return True

        # Check if instruments in track text
        for track in midi:
            if track.type == "text":
                words = {re.sub(pattern, '', word.lower())  for word in track.text.split()}
                if words.intersection(guitars):
                    has_guitar = True
                if words.intersection(drums):
                    has_drum = True
        if has_drum and has_guitar:
            return True

In [40]:
%time
file_processor = FileProcessor()
with Pool(6) as pools:
    print("Gettting started....")
    pools.map(file_processor.parse_file, os.listdir(raw_data_folder))

print(f"Processed {file_processor.num_processed} files in {raw_data_folder}.")
print(f"Encountered {file_processor.erors} errors.")
print("Number of songs already in data folder from a previous run:", len(file_processor.already_saved))
print("Number of new songs saved in data folder in this run", len(file_processor.saved_songs))
print("Total number of non-duplicate songs with drums and guitar saved:", len(file_processor.saved_songs) + len(already_saved))

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 12.2 µs
Gettting started....
